# 그룹연산과 변형

In [1]:
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [94]:
# 실습 예제1
df = pd.DataFrame({
    'city': ['부산', '부산', '부산', '부산', '서울', '서울', '서울'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})

In [3]:
# 실습 예제2
tips = pd.read_csv('examples/tips.csv')
tips = tips.sample(10)
tips

,total_bill,tip,smoker,day,time,size
68,20.23,2.01,No,Sat,Dinner,2
175,32.90,3.11,Yes,Sun,Dinner,2
77,27.20,4.00,No,Thur,Lunch,4
187,30.46,2.00,Yes,Sun,Dinner,5
227,20.45,3.00,No,Sat,Dinner,4
89,21.16,3.00,No,Thur,Lunch,2
198,13.00,2.00,Yes,Thur,Lunch,2
148,9.78,1.73,No,Thur,Lunch,2
159,16.49,2.00,No,Sun,Dinner,4
197,43.11,5.00,Yes,Thur,Lunch,4


## 그룹별 연산과 변형
* groupby()
* transform() : 스칼라 값이나 같은 크기를 가지는 배열을 반환하는 함수
* apply() : pandas 객체나 스칼라 값을 반환하는 함수
* apply : 분리 - 적용 - 병합
* 그룹 색인 생략하기
* 변위치 분석과 버킷 분석
* 그룹에 국한된 값으로 누락값 채우기

### add_prefix()

In [9]:
tip_means = tips.groupby('time').mean().add_prefix('mean_')
tip_means

,mean_total_bill,mean_tip,mean_size
time,,,
Dinner,24.106,2.424,3.4
Lunch,22.850,3.146,2.8


### groupby()

In [95]:
df

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4
4,서울,apple,150,5
5,서울,apple,200,6
6,서울,banana,400,7


In [96]:
df.groupby('city').mean()

,price,quantity
city,,
부산,212.5,2.5
서울,250.0,6.0


In [97]:
df.groupby(['city','fruits']).mean()

price  quantity
city fruits                 
부산   apple   100.0       1.0
     banana  275.0       3.5
     orange  200.0       2.0
서울   apple   175.0       5.5
     banana  400.0       7.0

In [100]:
# 'as_index=False' - groupby()의 결과값을 인덱스가 아닌 컬럼으로 반환
df.groupby(['city','fruits'], as_index=False).mean()

,city,fruits,price,quantity
0,부산,apple,100.0,1.0
1,부산,banana,275.0,3.5
2,부산,orange,200.0,2.0
3,서울,apple,175.0,5.5
4,서울,banana,400.0,7.0


In [103]:
# groupby() 내의 데이터 확인
df.groupby(['city','fruits']).get_group(('부산','banana'))

,city,fruits,price,quantity
2,부산,banana,250,3
3,부산,banana,300,4


In [104]:
# 그룹별 갯수를 반환
df.groupby(['city','fruits']).count()

price  quantity
city fruits                 
부산   apple       1         1
     banana      2         2
     orange      1         1
서울   apple       2         2
     banana      1         1

### groupby().agg(), groupby().aggregate()

In [106]:
df.groupby(['city']).agg(np.mean)

,price,quantity
city,,
부산,212.5,2.5
서울,250.0,6.0


In [107]:
def my_mean(s):
    return np.mean(s)
df.groupby('city').agg([my_mean, np.min, 'max'])

price           quantity         
     my_mean amin  max  my_mean amin max
city                                    
부산     212.5  100  300      2.5    1   4
서울     250.0  150  400      6.0    5   7

In [109]:
def my_mean(s):
    return np.mean(s)
df.groupby('city').agg({'price':my_mean, 'quantity':np.min})

,price,quantity
city,,
부산,212.5,1
서울,250.0,5


In [110]:
df

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4
4,서울,apple,150,5
5,서울,apple,200,6
6,서울,banana,400,7


### groupby().filter()

In [117]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])

def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")


df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

### groupby().transform()
transform()은 들어온 메서드에 대해 각 원소를 살리고 그 안에 연산결과를 채운다.

In [129]:
tips.groupby('day').transform(np.mean)

,total_bill,tip,size,pct
68,20.340000,2.505,3.000000,12.302833
175,26.616667,2.370,3.666667,9.382479
77,22.850000,3.146,2.800000,14.711118
187,26.616667,2.370,3.666667,9.382479
227,20.340000,2.505,3.000000,12.302833
89,22.850000,3.146,2.800000,14.711118
198,22.850000,3.146,2.800000,14.711118
148,22.850000,3.146,2.800000,14.711118
159,26.616667,2.370,3.666667,9.382479
197,22.850000,3.146,2.800000,14.711118


In [119]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])

display('df',"df.groupby('key').mean()",'df.groupby("key").transform(lambda x: x - x.mean())')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').mean()
     data1  data2
key              
A      1.5    4.0
B      2.5    3.5
C      3.5    6.0

df.groupby("key").transform(lambda x: x - x.mean())
   data1  data2
0   -1.5    1.0
1   -1.5   -3.5
2   -1.5   -3.0
3    1.5   -1.0
4    1.5    3.5
5    1.5    3.0

In [120]:
df.groupby('key').mean()

,data1,data2
key,,
A,1.5,4.0
B,2.5,3.5
C,3.5,6.0


In [123]:
display('df',"df.groupby('key').mean()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').mean()
     data1  data2
key              
A      1.5    4.0
B      2.5    3.5
C      3.5    6.0

In [124]:
df.groupby('key')['data2'].transform(np.mean)

0    4.0
1    3.5
2    6.0
3    4.0
4    3.5
5    6.0
Name: data2, dtype: float64

### groupby().apply()
apply() 함수는 그룹별로 정리된 결과값으로 출력된다.

In [12]:
# tips.groupby('day').transform(np.mean) # transform()과 apply() 비교
tips.groupby('day').apply(np.mean)

,total_bill,tip,size
day,,,
Sat,20.340000,2.505,3.000000
Sun,26.616667,2.370,3.666667
Thur,22.850000,3.146,2.800000


In [16]:
tips['pct'] = tips['tip']/tips['total_bill']*100
tips

,total_bill,tip,smoker,day,time,size,pct
68,20.23,2.01,No,Sat,Dinner,2,9.935739
175,32.90,3.11,Yes,Sun,Dinner,2,9.452888
77,27.20,4.00,No,Thur,Lunch,4,14.705882
187,30.46,2.00,Yes,Sun,Dinner,5,6.565988
227,20.45,3.00,No,Sat,Dinner,4,14.669927
89,21.16,3.00,No,Thur,Lunch,2,14.177694
198,13.00,2.00,Yes,Thur,Lunch,2,15.384615
148,9.78,1.73,No,Thur,Lunch,2,17.689162
159,16.49,2.00,No,Sun,Dinner,4,12.128563
197,43.11,5.00,Yes,Thur,Lunch,4,11.598237


In [20]:
def top(df, n=3, column='pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=3)

,total_bill,tip,smoker,day,time,size,pct
77,27.20,4.00,No,Thur,Lunch,4,14.705882
198,13.00,2.00,Yes,Thur,Lunch,2,15.384615
148,9.78,1.73,No,Thur,Lunch,2,17.689162


In [18]:
tips.sort_values(by='pct')[-6:]

,total_bill,tip,smoker,day,time,size,pct
159,16.49,2.00,No,Sun,Dinner,4,12.128563
89,21.16,3.00,No,Thur,Lunch,2,14.177694
227,20.45,3.00,No,Sat,Dinner,4,14.669927
77,27.20,4.00,No,Thur,Lunch,4,14.705882
198,13.00,2.00,Yes,Thur,Lunch,2,15.384615
148,9.78,1.73,No,Thur,Lunch,2,17.689162


In [21]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size        pct
smoker                                                            
No     227       20.45  3.00     No   Sat  Dinner     4  14.669927
       77        27.20  4.00     No  Thur   Lunch     4  14.705882
       148        9.78  1.73     No  Thur   Lunch     2  17.689162
Yes    175       32.90  3.11    Yes   Sun  Dinner     2   9.452888
       197       43.11  5.00    Yes  Thur   Lunch     4  11.598237
       198       13.00  2.00    Yes  Thur   Lunch     2  15.384615

In [24]:
tips.groupby('smoker').apply(top, n=3, column='total_bill')

total_bill   tip smoker   day    time  size        pct
smoker                                                            
No     227       20.45  3.00     No   Sat  Dinner     4  14.669927
       89        21.16  3.00     No  Thur   Lunch     2  14.177694
       77        27.20  4.00     No  Thur   Lunch     4  14.705882
Yes    187       30.46  2.00    Yes   Sun  Dinner     5   6.565988
       175       32.90  3.11    Yes   Sun  Dinner     2   9.452888
       197       43.11  5.00    Yes  Thur   Lunch     4  11.598237

In [2]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])

def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


In [7]:
df = pd.DataFrame({
    'city': ['부산', '부산', '부산', '부산', '서울', '서울', '서울'],
    'fruits': ['apple', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana'],
    'price': [100, 200, 250, 300, 150, 200, 400],
    'quantity': [1, 2, 3, 4, 5, 6, 7]
})
result = df.groupby(['city','fruits'], as_index=False).apply(lambda x: (x.price * x.quantity).sum())
display('df','result')

df
  city  fruits  price  quantity
0   부산   apple    100         1
1   부산  orange    200         2
2   부산  banana    250         3
3   부산  banana    300         4
4   서울   apple    150         5
5   서울   apple    200         6
6   서울  banana    400         7

result
  city  fruits   NaN
0   부산   apple   100
1   부산  banana  1950
2   부산  orange   400
3   서울   apple  1950
4   서울  banana  2800

In [13]:
result1 = df.groupby(['city','fruits'], as_index=False).sum()
result = df.groupby(['city','fruits'], as_index=False).sum().pivot('city','fruits')
display('df','result1','result')

,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,orange,200,2
2,부산,banana,250,3
3,부산,banana,300,4
4,서울,apple,150,5
5,서울,apple,200,6
6,서울,banana,400,7
,city,fruits,price,quantity
0,부산,apple,100,1
1,부산,banana,550,7


In [14]:
# 별도의 그룹 리스트(L)를 이용하여 DataFrame 조회 
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])

L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

In [17]:
# DataFrame의 인덱스를 dict와 결합하여 새롭게 그룹화하여 조회
df2 = df.set_index('key')
mapping = {'A':'vowel', 'B':'constant', 'C':'constant'}
df3 = df2.groupby(mapping).sum()
display('df2', 'df3')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
constant,12,19
vowel,3,8


In [19]:
df3 = df2.groupby(str.lower).mean()
display('df2','df3')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


In [23]:
mapping = {'A':'vowel', 'B':'constant', 'C':'constant'}
df3 = df2.groupby([str.lower, mapping]).mean()
display('df2','df3')

df2
     data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9

df3
            data1  data2
a vowel       1.5    4.0
b constant    2.5    3.5
c constant    3.5    6.0

In [32]:
import seaborn as sns
planets = sns.load_dataset('planets')

decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
result = planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)
display('planets','result')

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


## 그룹함수 적용의 색인 없애기
groupby() + apply() 연산의 결과값을 보면 index와 결과값안의 데이터가 중복된다. index의 출력여부를 group_keys로 설정한다.

In [25]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,pct
227,20.45,3.00,No,Sat,Dinner,4,14.669927
77,27.20,4.00,No,Thur,Lunch,4,14.705882
148,9.78,1.73,No,Thur,Lunch,2,17.689162
175,32.90,3.11,Yes,Sun,Dinner,2,9.452888
197,43.11,5.00,Yes,Thur,Lunch,4,11.598237
198,13.00,2.00,Yes,Thur,Lunch,2,15.384615


In [27]:
tips.groupby('smoker', group_keys =False).apply(top).describe()

,total_bill,tip,size,pct
count,6.000000,6.000000,6.000000,6.000000
mean,24.406667,3.140000,3.000000,13.916785
std,12.565514,1.225349,1.095445,2.928388
min,9.780000,1.730000,2.000000,9.452888
25%,14.862500,2.250000,2.000000,12.366159
50%,23.825000,3.055000,3.000000,14.687905
75%,31.475000,3.777500,4.000000,15.214932
max,43.110000,5.000000,4.000000,17.689162


## 변위치 분석과 버킷 분석
cut(), qcut() 메서드를 이용하여 선택한 크기만큼 혹은 표본 변위치에 따라 데이터 분리하고 groupby()와 조합하여 데이터 묶음에 대한 변위치 분석이나 버킷 분석

cut(), qcut() 계열의 매서드는 기본적으로 설정된 컬럼에 대한 sorting(정렬)을 진행한다.

### cut()
지정된 컬럼의 최소값과 최대값을 등분위로 나누는 메소드

In [30]:
factor_cut = pd.cut(tips['tip'], 4)
factor_cut

68     (1.727, 2.548]
175    (2.548, 3.365]
77     (3.365, 4.182]
187    (1.727, 2.548]
227    (2.548, 3.365]
89     (2.548, 3.365]
198    (1.727, 2.548]
148    (1.727, 2.548]
159    (1.727, 2.548]
197      (4.182, 5.0]
Name: tip, dtype: category
Categories (4, interval[float64]): [(1.727, 2.548] < (2.548, 3.365] < (3.365, 4.182] < (4.182, 5.0]]

In [32]:
tips.groupby(factor_cut).agg(['size','mean'])

total_bill             tip           size            pct  \
                     size       mean size      mean size      mean size   
tip                                                                       
(1.727, 2.548]          5  17.992000    5  1.948000    5  3.000000    5   
(2.548, 3.365]          3  24.836667    3  3.036667    3  2.666667    3   
(3.365, 4.182]          1  27.200000    1  4.000000    1  4.000000    1   
(4.182, 5.0]            1  43.110000    1  5.000000    1  4.000000    1   

                           
                     mean  
tip                        
(1.727, 2.548]  12.340813  
(2.548, 3.365]  12.766836  
(3.365, 4.182]  14.705882  
(4.182, 5.0]    11.598237

### qcut()
주어진 데이터의 갯수를 n등분하는 메서드

In [52]:
factor_qcut = pd.qcut(tips.total_bill, 5, labels=False)
factor_qcut
factor_qcut.value_counts()

0    2
1    2
2    2
3    2
4    2
Name: total_bill, dtype: int64

In [56]:
factor_qcut = pd.qcut(tips.total_bill, 5, labels=False)
factor_qcut
# factor_qcut.value_counts()

68     1
175    4
77     3
187    3
227    2
89     2
198    0
148    0
159    1
197    4
Name: total_bill, dtype: int64

In [57]:
tips.groupby(factor_qcut).agg(['size','mean'])

total_bill          tip        size       pct           
                 size    mean size   mean size mean size       mean
total_bill                                                         
0                   2  11.390    2  1.865    2  2.0    2  16.536888
1                   2  18.360    2  2.005    2  3.0    2  11.032151
2                   2  20.805    2  3.000    2  3.0    2  14.423810
3                   2  28.830    2  3.000    2  4.5    2  10.635935
4                   2  38.005    2  4.055    2  3.0    2  10.525562

In [59]:
# tits.tip 에 같은 값이 있는 경우 qcut()의 동일한 개수를 반환하지 않는다.
factor_qcut = pd.qcut(tips.tip, 5, labels=False)
tips.groupby(factor_qcut).agg(['size','mean'])

total_bill             tip         size            pct           
          size       mean size    mean size      mean size       mean
tip                                                                  
0            4  17.432500    4  1.9325    4  3.250000    4  12.942082
2            3  20.613333    3  2.6700    3  2.666667    3  12.927786
3            1  32.900000    1  3.1100    1  2.000000    1   9.452888
4            2  35.155000    2  4.5000    2  4.000000    2  13.152060

## 그룹에 국한된 값으로 누락값 채우기

### fillna()

In [60]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.788156
2         NaN
3    1.075705
4         NaN
5   -0.420205
dtype: float64

In [61]:
# s의 평균으로 Null 값을 대체
s.fillna(s.mean())

0    0.481218
1    0.788156
2    0.481218
3    1.075705
4    0.481218
5   -0.420205
dtype: float64

In [65]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.686776
New York     -1.067945
Vermont            NaN
Florida      -0.222128
Oregon       -1.047117
Nevada             NaN
California   -0.758808
Idaho              NaN
dtype: float64

In [66]:
data.groupby(group_key).mean()

East   -0.658950
West   -0.902963
dtype: float64

In [74]:
fill_mean = lambda x: x.fillna(x.mean())
data.groupby(group_key).apply(fill_mean)
# data.groupby(group_key).transform(fill_mean) # 위와 동일

Ohio         -0.686776
New York     -1.067945
Vermont      -0.658950
Florida      -0.222128
Oregon       -1.047117
Nevada       -0.902963
California   -0.758808
Idaho        -0.902963
dtype: float64

In [75]:
data.groupby(group_key).transform(np.mean)

Ohio         -0.658950
New York     -0.658950
Vermont      -0.658950
Florida      -0.658950
Oregon       -0.902963
Nevada       -0.902963
California   -0.902963
Idaho        -0.902963
dtype: float64

In [76]:
# 그룹에 따라 정의된 값 채우기
fill_values = {'East':0.5, 'West':-1}
fill_func = lambda x: x.fillna(fill_values[x.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.686776
New York     -1.067945
Vermont       0.500000
Florida      -0.222128
Oregon       -1.047117
Nevada       -1.000000
California   -0.758808
Idaho        -1.000000
dtype: float64

## 피벗테이블과 교차 일람표


### 교차 일람표 - pivot_table()

In [77]:
tips

,total_bill,tip,smoker,day,time,size,pct
68,20.23,2.01,No,Sat,Dinner,2,9.935739
175,32.90,3.11,Yes,Sun,Dinner,2,9.452888
77,27.20,4.00,No,Thur,Lunch,4,14.705882
187,30.46,2.00,Yes,Sun,Dinner,5,6.565988
227,20.45,3.00,No,Sat,Dinner,4,14.669927
89,21.16,3.00,No,Thur,Lunch,2,14.177694
198,13.00,2.00,Yes,Thur,Lunch,2,15.384615
148,9.78,1.73,No,Thur,Lunch,2,17.689162
159,16.49,2.00,No,Sun,Dinner,4,12.128563
197,43.11,5.00,Yes,Thur,Lunch,4,11.598237


In [78]:
# tips.groupby(['day','smoker']).mean()  # 아래와 동일
tips.pivot_table(index=['day','smoker'])

pct      size    tip  total_bill
day  smoker                                        
Sat  No      12.302833  3.000000  2.505      20.340
Sun  No      12.128563  4.000000  2.000      16.490
     Yes      8.009438  3.500000  2.555      31.680
Thur No      15.524246  2.666667  2.910      19.380
     Yes     13.491426  3.000000  3.500      28.055

In [84]:
tips.pivot_table(['tip','size'], index=['day','smoker'], columns='time')

size              tip      
time        Dinner     Lunch Dinner Lunch
day  smoker                              
Sat  No        3.0       NaN  2.505   NaN
Sun  No        4.0       NaN  2.000   NaN
     Yes       3.5       NaN  2.555   NaN
Thur No        NaN  2.666667    NaN  2.91
     Yes       NaN  3.000000    NaN  3.50

In [86]:
tips

,total_bill,tip,smoker,day,time,size,pct
68,20.23,2.01,No,Sat,Dinner,2,9.935739
175,32.90,3.11,Yes,Sun,Dinner,2,9.452888
77,27.20,4.00,No,Thur,Lunch,4,14.705882
187,30.46,2.00,Yes,Sun,Dinner,5,6.565988
227,20.45,3.00,No,Sat,Dinner,4,14.669927
89,21.16,3.00,No,Thur,Lunch,2,14.177694
198,13.00,2.00,Yes,Thur,Lunch,2,15.384615
148,9.78,1.73,No,Thur,Lunch,2,17.689162
159,16.49,2.00,No,Sun,Dinner,4,12.128563
197,43.11,5.00,Yes,Thur,Lunch,4,11.598237


In [89]:
tips.pivot_table(['tip','size'], index=['day','smoker'], columns='time', margins=True)

size                        tip              
time        Dinner     Lunch       All Dinner  Lunch    All
day  smoker                                                
Sat  No        3.0       NaN  3.000000  2.505    NaN  2.505
Sun  No        4.0       NaN  4.000000  2.000    NaN  2.000
     Yes       3.5       NaN  3.500000  2.555    NaN  2.555
Thur No        NaN  2.666667  2.666667    NaN  2.910  2.910
     Yes       NaN  3.000000  3.000000    NaN  3.500  3.500
All            3.4  2.800000  3.100000  2.424  3.146  2.785

In [90]:
tips.pivot_table(['tip','size'], index=['day','smoker'], columns='time', margins=True,fill_value=0)

size                        tip              
time        Dinner     Lunch       All Dinner  Lunch    All
day  smoker                                                
Sat  No        3.0  0.000000  3.000000  2.505  0.000  2.505
Sun  No        4.0  0.000000  4.000000  2.000  0.000  2.000
     Yes       3.5  0.000000  3.500000  2.555  0.000  2.555
Thur No        0.0  2.666667  2.666667  0.000  2.910  2.910
     Yes       0.0  3.000000  3.000000  0.000  3.500  3.500
All            3.4  2.800000  3.100000  2.424  3.146  2.785

In [91]:
tips.pivot_table(['tip','size'], index=['day','smoker'], columns='time', margins=True,fill_value=0, aggfunc='sum')


size              tip              
time        Dinner Lunch All Dinner  Lunch    All
day  smoker                                      
Sat  No          6     0   6   5.01   0.00   5.01
Sun  No          4     0   4   2.00   0.00   2.00
     Yes         7     0   7   5.11   0.00   5.11
Thur No          0     8   8   0.00   8.73   8.73
     Yes         0     6   6   0.00   7.00   7.00
All             17    14  31  12.12  15.73  27.85

### 교차 일람표 - crosstab()
그룹 빈도를 개산하기 위한 피벗 테이블의 특수한 경우

In [93]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker       No  Yes  All
time   day               
Dinner Sat    2    0    2
       Sun    1    2    3
Lunch  Thur   3    2    5
All           6    4   10

### merge()

In [41]:
 file_path='examples/drinks.csv'
 drinks = pd.read_csv(file_path)
 drinks['continent'] = drinks['continent'].fillna('OT')

 result = drinks.groupby('continent').mean()['wine_servings']
 df = result.to_frame().reset_index()
 df = df.rename(columns={'wine_servings':'wine_servings_cont_avg'})
 df = pd.merge(drinks, df, on='continent', how='outer')
 display('drinks','df')

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA
189,Vietnam,111,2,1,2.0,AS
190,Yemen,6,0,0,0.1,AS
191,Zambia,32,19,4,2.5,AF


In [39]:
# 위의 전체 과정을 한줄로 바꿀수 있음
drinks['wine_servings_cont_avg'] = drinks.groupby('continent')['wine_servings'].transform(np.mean)
drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,wine_servings_cont_avg
0,Afghanistan,0,0,0,0.0,AS,9.068182
1,Albania,89,132,54,4.9,EU,142.222222
2,Algeria,25,0,14,0.7,AF,16.264151
3,Andorra,245,138,312,12.4,EU,142.222222
4,Angola,217,57,45,5.9,AF,16.264151
...,...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA,62.416667
189,Vietnam,111,2,1,2.0,AS,9.068182
190,Yemen,6,0,0,0.1,AS,9.068182
191,Zambia,32,19,4,2.5,AF,16.264151
